# 🛰️ LandCover Classification

Este notebook realiza a classificação de uso e cobertura da terra utilizando imagens Sentinel-2 e Random Forest.

## 🔧 Importação de Bibliotecas

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import rasterio
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from src.preprocessing import load_image, normalize_image, calculate_ndvi, load_labels
from src.model import train_model, classify_image
from src.export import export_raster
from src.visualization import plot_classification
    

## 📂 Carregamento dos Dados

In [ ]:

image_path = "data/sentinel2_example.tif"
label_path = "data/labels.tif"

image, profile = load_image(image_path)
labels, _ = load_labels(label_path)

print("Imagem shape:", image.shape)
print("Labels shape:", labels.shape)
    

## 🧽 Pré-processamento e Cálculo de NDVI

In [ ]:

image_normalized = normalize_image(image)

if image.shape[0] >= 5:
    ndvi = calculate_ndvi(image_normalized[4], image_normalized[3])
    image_with_ndvi = np.vstack([image_normalized, ndvi[None, ...]])
else:
    print("NDVI não pôde ser calculado. Prosseguindo sem NDVI.")
    image_with_ndvi = image_normalized
    

## 🏷️ Preparação dos Dados para o Modelo

In [ ]:

labels_flat = labels.flatten()
mask = labels_flat != 0

features = image_with_ndvi.reshape(image_with_ndvi.shape[0], -1).T
features = features[mask]
labels_flat = labels_flat[mask]

X_train, X_test, y_train, y_test = train_test_split(
    features, labels_flat, test_size=0.3, random_state=42
)
    

## 🧠 Treinamento do Modelo

In [ ]:

model = train_model(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%")
    

## 🗺️ Classificação da Imagem Completa

In [ ]:

classified = classify_image(model, image_with_ndvi)
    

## 💾 Exportação dos Resultados

In [ ]:

export_raster("outputs/rasters/resultado_classificacao.tif", classified, profile)
plot_classification(classified, "outputs/figures/mapa_classificado.pdf")
    

## ✅ Fim do Pipeline

In [ ]:

print("Pipeline executado com sucesso!")
    